In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from MCMAC_pre_merger import MCengine_pre_merger
from MCMAC_post_merger import MCengine_post_merger 

import numpy as np
import pickle

# Input A1758 parameters

In [3]:
# Setting simulation parameters (define prior distributions)
N_mc = 1000  # Number of Monte Carlo samples

arcsecond = 96
redshift = 0.2799

# (mean value, standard deviation)
M1 = (7.90e14, 1.89e14)  # Mass distribution of A1758NW (Subcluster 1)
M2 = (5.49e14, 1.67e14)  # Mass distribution of A1758NE (Subcluster 2)
Z1 = (0.2799, 0.0006)  # Redshift distribution of A1758NW (Subcluster 1)
Z2 = (0.2780, 0.0007)  # Redshift distribution of A1758NE (Subcluster 2)

TSC_simulation = 0.27
TSC_simulation_lower = 0
TSC_simulation_upper = 0

# when you use pre-MCMAC code
# prefix = "../RealClusters/A1758/simulation_pre_output" 

# when you use post-MCMAC code
prefix = "../RealClusters/A1758/simulation_post_output" 

result_file = "result_A1758.csv"

## arcsecond to kpc

In [3]:
from astropy.cosmology import Planck18 as cosmo

def arcsecondToKpc(redshift, arcsecond):
    # Define the redshift and angle in arcseconds
    redshift = 0.2799
    arcsecond = 96

    # Convert arcseconds to kiloparsecs
    kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(redshift).value / 60  # kpc per arcsec
    distance_kpc = arcsecond * kpc_per_arcsec
    return distance_kpc

In [4]:

distance_Mpc = arcsecondToKpc(redshift, arcsecond) * 0.001 # kpc to Mpc

D_proj = (distance_Mpc, 0.05*distance_Mpc, 0.05*distance_Mpc)  # Projected separation distance distribution (mean 408 kpc, standard deviation 14 kpc)

## Confirm it collision or not

In [5]:
# Execute simulation (derive posterior distributions of dynamical properties such as collision velocity, time since collision, etc.)
# MCengine_pre_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Probability of system being bound = 0.99200

Probability of system being boud is over 50%, then use post merger code.

In [6]:
MCengine_post_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Completed Monte Carlo iteration 10 of 1000.
~4 minutes remaining
Completed Monte Carlo iteration 100 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 1000 of 1000.
~0 minutes remaining


(array([1.05874260e+15, 7.98119045e+14, 7.55269231e+14, 6.11256829e+14,
        7.41200079e+14, 5.63867269e+14, 8.57710562e+14, 8.05394290e+14,
        6.56349289e+14, 7.37518980e+14, 9.11767424e+14, 9.50200974e+14,
        7.14382684e+14, 5.83605559e+14, 8.79193180e+14, 6.72280542e+14,
        8.86892539e+14, 8.11729396e+14, 1.04255109e+15, 6.90068152e+14,
        8.30233679e+14, 8.91874045e+14, 5.65238156e+14, 9.72874531e+14,
        8.90695336e+14, 8.97410947e+14, 6.29766893e+14, 7.68842823e+14,
        9.54348815e+14, 9.88502063e+14, 7.92888005e+14, 6.75168221e+14,
        5.15074065e+14, 9.16386887e+14, 7.43964499e+14, 6.33385518e+14,
        9.42126331e+14, 1.00636155e+15, 1.03018001e+15, 7.36557095e+14,
        8.51683471e+14, 7.34963161e+14, 8.88208666e+14, 1.03118942e+15,
        7.13933674e+14, 7.25821774e+14, 6.27023555e+14, 5.73005209e+14,
        9.58336936e+14, 9.86536191e+14, 1.01153928e+15, 1.67370620e+14,
        5.56072887e+14, 9.04805006e+14, 7.96742177e+14, 9.591240

In [5]:
import pickle
import numpy as np
import pandas as pd

# 単位の定義
units = {
    'm1': '10^14 M☉',
    'm2': '10^14 M☉',
    'z1': '',
    'z2': '',
    'dproj': 'Mpc',
    'vradobs': 'km s^{-1}',
    'alpha': 'degree',
    'v3dobs': 'km s^{-1}',
    'd3d': 'Mpc',
    'v3dcol': 'km s^{-1}',
    'dmax': 'Mpc',
    'TSM0': 'Gyr',
    'TSM1': 'Gyr',
    'T': 'Gyr',
    'prob': ''
}

# パラメータファイルのリスト
parameters = ['_m_1.pickle', '_m_2.pickle', '_z_1.pickle', '_z_2.pickle', '_d_proj.pickle', 
              '_v_rad_obs.pickle', '_alpha.pickle', '_v_3d_obs.pickle', '_d_3d.pickle', 
              '_v_3d_col.pickle', '_d_max.pickle', '_TSM_0.pickle', '_TSM_1.pickle', 
              '_T.pickle', '_prob.pickle']

results = []

for param in parameters:
    with open(prefix + param, 'rb') as f:
        data = pickle.load(f)
        if param in ['_m_1.pickle', '_m_2.pickle']:
            # 質量をe14単位に変換
            data = np.array(data) / 1e14
        
        # Medianを計算
        median = np.median(data)
        
        # 68%および95%の信頼区間を計算
        lcl_68, ucl_68 = np.percentile(data, [16, 84])
        lcl_95, ucl_95 = np.percentile(data, [2.5, 97.5])
        
        param_name = param.replace('_', '').replace('.pickle', '')
        
        # 数値を有効数字2桁にフォーマット
        if param_name not in ['z_1', 'z_2']:
            median = round(median, 2)
            lcl_68 = round(lcl_68, 2)
            ucl_68 = round(ucl_68, 2)
            lcl_95 = round(lcl_95, 2)
            ucl_95 = round(ucl_95, 2)
        
        results.append({
            'Parameter': param_name,
            'Units': units[param_name],
            'Median': median,
            '68% LCL': lcl_68,
            '68% UCL': ucl_68,
            # '95% LCL': lcl_95,
            # '95% UCL': ucl_95
        })

# 結果をDataFrameに変換
df_results = pd.DataFrame(results)

display(df_results)

# TSM0とTSM1に対応する行の抽出
tsm0_row = df_results[df_results['Parameter'] == 'TSM0']
tsm1_row = df_results[df_results['Parameter'] == 'TSM1']

# CSVに出力するための新しい行を作成
new_row = {
    'ClusterName': 'A1758',
    'MCMAC_TSC0': tsm0_row['Median'].values[0],
    'MCMAC_TSC0.lower': tsm0_row['68% LCL'].values[0],
    'MCMAC_TSC0.upper': tsm0_row['68% UCL'].values[0],
    'MCMAC_TSC1': tsm1_row['Median'].values[0],
    'MCMAC_TSC1.lower': tsm1_row['68% LCL'].values[0],
    'MCMAC_TSC1.upper': tsm1_row['68% UCL'].values[0],
    'Simulation_TSC': 0.27, 
    'Simulation_TSC.lower': 0,
    'Simulation_TSC.upper': 0
}

# 新しい行をDataFrameに変換
new_row_df = pd.DataFrame([new_row])

# CSVファイルに出力
new_row_df.to_csv(result_file, index=False)

,Parameter,Units,Median,68% LCL,68% UCL
0,m1,10^14 M☉,7.98,6.19,9.82
1,m2,10^14 M☉,5.53,3.92,7.09
2,z1,,0.28,0.28,0.28
3,z2,,0.28,0.28,0.28
4,dproj,Mpc,0.42,0.40,0.44
5,vradobs,km s^{-1},426.56,213.31,635.90
6,alpha,degree,39.98,13.84,69.93
7,v3dobs,km s^{-1},699.31,363.53,1779.24
8,d3d,Mpc,0.55,0.44,1.23
9,v3dcol,km s^{-1},1780.15,1412.32,2696.69
